# Stepshifter using Darts

## Introduction to stepshifter
Suppose we have data ranging from $ \tau_0 $ to $ \tau_f $, where $ \tau_0 $ is the first observation available for training, and $ \tau_f $ is the last observation. We create three partitions: 
1. Calibration partition from $ \tau_0 $ to $ \tau_c $, 

2. Testing partition from $ \tau_0 $ to $ \tau_t $,  

3. Forecasting partition from $ \tau_0 $ to $ \tau_f $,
 
where $\tau_c < \tau_t < \tau_f$.

For calibration and testing partition, the period is further divided into two parts: train and predict. 

For forecasting partition, true forecasts are generated with number of months into the future. We refer these as steps **_s_** $ \in [1, 36]$.

| **Partition**       | **Train**             | **Predict**               |
|---------------------|-----------------------|---------------------------|
| **Calibration**     | $ [\tau_0, \tau_c'] $ | $ [\tau_c' + 1, \tau_c] $ |
| **Testing**         | $ [\tau_0, \tau_t'] $ | $ [\tau_t' + 1, \tau_t] $ |
| **Forecasting**     | $ [\tau_0, \tau_f] $  | $ [\tau_f + 1, \tau_f + 1 + s_{max}] $   |


For each step $s \in [1, 36]$, we do the following prodedure:
1. Train the model on the calibration train period. 

2. Generate predictions for the calibration predict period and evaluate the model. 

3. Repeat step 1 and 2 by tuning hyperparameters.

4. Train the model on the testing train period with parameters from previous step. 

5. Generate predictions for the testing predict period and evaluate the model again.

6. Train the model on the forecasting train period, i.e., the whole observation period. 

7. Generate true diagonal predictions for the forecasting predict period.

## Building Stepshifter using Darts

We build stepshifter models using the library ```darts```. To get started, let’s first import the required modules.

### 1. Loading Required Libraries

In [16]:
import pandas as pd
import numpy as np
from darts.models import LightGBMModel
from darts import TimeSeries
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

### 2. Create a dummy views dataset

In [218]:
target = 'ln_geb_sb_dep'
level = 'country_id'
time = 'month_id'
covariates = ['decay_ged_sb', 'ln_pop_gpw_sum']
calibration_partition = {'train': (100, 120),
                         'predict': (121, 125)}

In [219]:
idx = pd.MultiIndex.from_product([[*range(100, 126)], [1, 2, 3, 4, 5]], names=['month_id', 'country_id'])
df = pd.DataFrame(index=idx)
df[target] = np.abs(np.random.normal(0, 1, len(df)))
df[covariates[0]] = np.abs(np.random.normal(0, 10, len(df)))
df[covariates[1]] = np.abs(np.random.normal(0, 10, len(df)))
df

ln_geb_sb_dep  decay_ged_sb  ln_pop_gpw_sum
month_id country_id                                             
100      1                0.093158     14.656739        9.106451
         2                0.510989     17.690503       12.710653
         3                2.553550      3.667464        9.800623
         4                0.168385      3.838747       16.178006
         5                0.326012      2.022804        3.807781
...                            ...           ...             ...
125      1                0.628144      6.534629        4.679504
         2                0.098934      3.865416       11.702807
         3                0.338357      6.355411       11.140814
         4                0.992578      2.083225        6.309308
         5                0.020417      3.603687        9.420602

[130 rows x 3 columns]

### 3. Train stepshifter

The default **steps** is from 1 to 36, which means we train 36 models and each of them is trained with targets *s* month ahead of covariates ($s \in [1, 36] $). In this example, we set step as 2 for simplicity, meaning that we are predicting two months into the future.

#### Pre-process the data to match the format

In [245]:
# reset index to make sure the level (country_id/priogrid_gid) is in the columns
df_reset = df.reset_index(level=[1])

# create a TimeSeries object from the dataframe grouping by the level. Series is a list of TimeSeries objects (one for each country or priogrid)
target_series = TimeSeries.from_group_dataframe(df_reset, group_cols=level, value_cols=target)
target_series_train = TimeSeries.from_group_dataframe(df_reset.loc[:calibration_partition['train'][1]], group_cols=level, value_cols=target)
covariates_series = TimeSeries.from_group_dataframe(df_reset, group_cols=level, value_cols=covariates)
covariates_series_train = TimeSeries.from_group_dataframe(df_reset.loc[:calibration_partition['train'][1]], group_cols=level, value_cols=covariates)

#### Fit the stepshifter model

In [221]:
model = LightGBMModel(lags_past_covariates=[-2])
model.fit(target_series_train, past_covariates=covariates_series_train)

LightGBMModel(lags=None, lags_past_covariates=[-2], lags_future_covariates=None, output_chunk_length=1, output_chunk_shift=0, add_encoders=None, likelihood=None, quantiles=None, random_state=None, multi_models=True, use_static_covariates=True, categorical_past_covariates=None, categorical_future_covariates=None, categorical_static_covariates=None)

### 4. Predict (evaluate)

For calibration and testing partition, we use part of the observed data to predict the rest. For every spatial unit, all the months in the predict period have 36 columns, 
step_1, step_2, ..., step_36, each of which is generated by one of the 36 models correspondingly. Then we'll compared the predictions and the actual values with some metrics.

Darts automatically locates the time period of past_covariates. We don't need to manually align past_covariates with the series; darts handles this internally.

In [222]:
horizon = calibration_partition['predict'][1] - calibration_partition['predict'][0] + 1
ts_pred = model.predict(n=horizon,
                        series=target_series_train,
                        past_covariates=covariates_series,
                        show_warnings=False)

ts_pred is a list of timeseries data array (one for each country or priogrid). To allow for views_forecasts, we need to convert to the standrad format.

In [223]:
ts_pred[1]

<TimeSeries (DataArray) (month_id: 5, component: 1, sample: 1)>
array([[[0.74208916]],

       [[0.60084986]],

       [[0.78544205]],

       [[0.98202764]],

       [[0.80826813]]])
Coordinates:
  * month_id   (month_id) int64 121 122 123 124 125
  * component  (component) object 'ln_geb_sb_dep'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  country_id\ncomponent             ...
    hierarchy:          None

In [224]:
index_tuples, df_list = [], []
test_period = slice(calibration_partition['predict'][0], calibration_partition['predict'][1])

for pred in ts_pred:
    df_pred = pred.pd_dataframe().loc[test_period]
    level_ = int(pred.static_covariates.iat[0, 0])
    index_tuples.extend([(month_, level_) for month_ in df_pred.index])
    df_list.append(df_pred.values)

df_preds = pd.DataFrame(
    data=np.concatenate(df_list),
    index=pd.MultiIndex.from_tuples(index_tuples, names=[time, level]),
    columns=[f"step_pred_2"]
)
df_preds.sort_index()

step_pred_2
month_id country_id             
121      1              0.650868
         2              0.742089
         3              0.591931
         4              0.414725
         5              0.660234
122      1              0.982028
         2              0.600850
         3              0.507358
         4              0.912546
         5              0.462735
123      1              0.665042
         2              0.785442
         3              0.801316
         4              0.526126
         5              0.819441
124      1              0.665042
         2              0.982028
         3              0.760936
         4              0.951254
         5              0.713506
125      1              1.083709
         2              0.808268
         3              0.338954
         4              0.412615
         5              0.939377

### 5. Predict (true forecasts)

When using forecasting partition, we use all the observed data to train the model. Instead of generating 36 predictions for each month in the predict period, we use the last-month-with-data 

and only the model with the correct step to forecast each month one prediction for every spatial unit for every month. This is essentially taking the diagonal of the the step-specific results.

Let's set steps from 1 to 2 and train 2 models to see how true forecasts work.

In [250]:
forecasting_partition = {'train': (100, 125),
                         'predict': (126, 127)} # the length of the predict period should be the same as max(steps)
steps = [1, 2]
models = {}
pred_by_step = []

In [251]:
for step in steps:
    model = LightGBMModel(lags_past_covariates=[-step])
    model.fit(target_series, past_covariates=covariates_series)
    models[step] = model

In [252]:
for step in steps:
    model = models[step]
    ts_pred = model.predict(n=step,
                        series=target_series,
                        past_covariates=covariates_series,
                        show_warnings=False)
    pred_by_step.append(ts_pred)

You can see the model trained with step==1 can only generate prediction for month 126 which uses covariates from month 125. The model trained with step==2 generate predictions for both month 

126 and 127, and for month 126 it uses covariates from month 124. Since we only need predictions from the last available month, we take the former prediction for month 126.

In [256]:
pred_by_step[0][0]

<TimeSeries (DataArray) (month_id: 1, component: 1, sample: 1)>
array([[[0.95919923]]])
Coordinates:
  * month_id   (month_id) int64 126
  * component  (component) object 'ln_geb_sb_dep'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  country_id\ncomponent             ...
    hierarchy:          None

In [255]:
pred_by_step[1][0]

<TimeSeries (DataArray) (month_id: 2, component: 1, sample: 1)>
array([[[0.86002896]],

       [[0.39683682]]])
Coordinates:
  * month_id   (month_id) int64 126 127
  * component  (component) object 'ln_geb_sb_dep'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  country_id\ncomponent             ...
    hierarchy:          None

In [ ]:
horizon = 2
ts_pred = model.predict(n=horizon,
                        series=target_series,
                        past_covariates=covariates_series,
                        show_warnings=False)

The code to process true forecast is a bit different from evaluate.

In [235]:
ts_pred[0]

<TimeSeries (DataArray) (month_id: 2, component: 1, sample: 1)>
array([[[0.86002896]],

       [[0.39683682]]])
Coordinates:
  * month_id   (month_id) int64 126 127
  * component  (component) object 'ln_geb_sb_dep'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  country_id\ncomponent             ...
    hierarchy:          None

In [234]:
index_tuples, df_list = [], []
for pred in ts_pred:
    df_pred = pred.pd_dataframe()
    level_ = int(pred.static_covariates.iat[0, 0])
    index_tuples.extend([(month_, level_) for month_ in df_pred.index])
    df_list.append(df_pred.values)

df_preds = pd.DataFrame(
    data=np.concatenate(df_list),
    index=pd.MultiIndex.from_tuples(index_tuples, names=[time, level]),
    columns=["step_pred_combined"]
)

df_preds.sort_index()

step_pred_combined
month_id country_id                    
126      1                     0.860029
         2                     1.017289
         3                     0.951714
         4                     1.025751
         5                     0.973283
127      1                     0.396837
         2                     0.746897
         3                     0.965726
         4                     0.716019
         5                     0.353045

In [110]:
steps = [1, 2]
partition_train = (100, 110)
partition_pred = (111, 112)
models = {}
for step in steps:
    model = LightGBMModel(lags_past_covariates=[-step])
    model.fit(target_series, past_covariates=covariates_series)
    models[step] = model

In [116]:
def predict_by_step_combined(model, step, target_series, covariates_series):
    ts_pred = model.predict(n=step,
                            series=target_series,
                            past_covariates=covariates_series,
                            show_warnings=False)
    index_tuples, df_list = [], []
    for pred in ts_pred:
        df_pred = pred.pd_dataframe().loc[[partition_pred[0] + step - 1]]
        level_ = int(pred.static_covariates.iat[0, 0])
        index_tuples.extend([(month_, level_) for month_ in df_pred.index])
        df_list.append(df_pred.values)

    df_preds = pd.DataFrame(
        data=np.concatenate(df_list),
        index=pd.MultiIndex.from_tuples(index_tuples, names=[time, level]),
        columns=["step_pred_combined"]
    )

    return df_preds.sort_index()

In [117]:
pred_by_step = [predict_by_step_combined(models[step], 
                                         step, 
                                         target_series, 
                                         covariates_series) for step in steps]
pred = pd.concat(pred_by_step, axis=0)

In [118]:
pred

step_pred_combined
month_id country_id                    
111      1                     0.620340
         2                     0.484488
         3                     0.593784
         4                     0.709319
         5                     0.743413
112      1                     0.673457
         2                     0.452722
         3                     0.473049
         4                     0.846532
         5                     0.589733